In [ ]:
%config InlineBackend.figure_format = 'svg'
from datetime import datetime
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from skylark import skylark_root, MB, GB
from skylark.gateway.chunk import ChunkState

base_dir = skylark_root / "data" / "experiments" / "benchmark_triangles"
fig_dir = base_dir / "figures" / "benchmark_triangles" / datetime.now().strftime("%Y%m%d_%H%M%S")
fig_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
configs = list((base_dir / "logs").glob("*"))

rows = []
for config in configs:
    for f in (skylark_root / "data" / "experiments" / "benchmark_triangles" / config / "results").glob("*.pkl"):
        with open(f, "rb") as fp:
            result = pickle.load(fp)
        rows.append(result)
df = pd.DataFrame(rows)
df_orig = df.copy()

if "error" in df.columns:
    for row in df[df.error.notnull()].itertuples():
        print(f"{row.src_region} -> {row.dst_region} -> {row.inter_region}: {row.error}")
    df = df[df.error.isnull()]
df

In [ ]:
if False:
    for (src, dst), group_df in df.groupby(["src_region", "dst_region"]):
        group_df = group_df.reset_index(drop=True)
        group_df = group_df[(group_df.inter_region != src) & (group_df.inter_region != dst)]

        # sort by throughput_gbits
        group_df = group_df.sort_values(by="throughput_gbits", ascending=True)
        group_df["inter_region"] = group_df.inter_region.fillna("Direct")
        group_df["label"] = group_df.apply(lambda row: f"{row.inter_region} ({row.num_gateways} x {row.num_outgoing_connections})", axis=1)
        group_df["transfer_bytes"] = group_df.apply(lambda row: row.n_chunks * row.chunk_size_mb * MB, axis=1)
        assert np.all(group_df.transfer_bytes == group_df.transfer_bytes[0])

        # plot barh
        # title = f"Skylark throughput {src} => {dst}, {group_df.transfer_bytes[0] / GB:.2f}GB in group_df.num_chunks[0]x group_df.chunk_size_mb[0]MB chunks"
        with plt.style.context(["seaborn-white", {"font.size": 10}]):
            fig, ax = plt.subplots(figsize=(8, 6))
            ax.barh(
                group_df.label,
                group_df.throughput_gbits,
            )
            ax.set_xlabel("Throughput (Gbps)")
            ax.set_ylabel("Inter-region relay")
            ax.set_title(f"{src} => {dst}, {group_df.transfer_bytes[0] / GB:.2f}GB ({group_df.n_chunks[0]}x{group_df.chunk_size_mb[0]}MB)")

            # add white labels to each bar
            for i, (label, throughput) in enumerate(zip(group_df.label, group_df.throughput_gbits)):
                # assert i, throughput not NaN
                if not np.isnan(throughput):
                    ax.text(
                        throughput,
                        i,
                        f"{throughput:.2f} Gbps ",
                        ha="right",
                        va="center",
                        color="white",
                    )

            # save plots
            fig.tight_layout()
            fig.savefig(fig_dir / f"{src}_{dst}.png", dpi=300, bbox_inches="tight")
            fig.savefig(fig_dir / f"{src}_{dst}.pdf", dpi=300, bbox_inches="tight")

In [ ]:
# vectorized version
def compute_throughput_df(df_log, chunk_size_bytes):
    df_log = df_log.reset_index(drop=True).set_index(["path_idx", "hop_idx", "chunk_id", "state"]).sort_index()
    unstacked_time = df_log.time.unstack(level=3)
    unstacked_time.columns = [f"{col.name}_time" for col in unstacked_time.columns]
    unstacked_time["runtime_upload"] = (
        unstacked_time["upload_complete_time"] - unstacked_time["upload_in_progress_time"]
    ).dt.total_seconds()
    unstacked_time["runtime_download"] = (
        unstacked_time["downloaded_time"] - unstacked_time["download_in_progress_time"]
    ).dt.total_seconds()
    unstacked_time["runtime_queue"] = (unstacked_time["upload_in_progress_time"] - unstacked_time["upload_queued_time"]).dt.total_seconds()
    unstacked_time["runtime_total"] = (unstacked_time["upload_complete_time"] - unstacked_time["registered_time"]).dt.total_seconds()
    unstacked_time["upload_gbits"] = (chunk_size_bytes / GB * 8) / unstacked_time["runtime_upload"]
    unstacked_time["download_gbits"] = (chunk_size_bytes / GB * 8) / unstacked_time["runtime_download"]
    return unstacked_time

In [ ]:
# plot grid of subplots
grouped = df.groupby(["src_region", "dst_region"])
n_rows = len(grouped.groups)
n_cols = max(len(grouped.get_group(group).inter_region.unique()) for group in grouped.groups)
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols * 4, n_rows * 4))
with tqdm(total=n_rows * n_cols) as pbar:
    for plot_row_idx, ((src_region, dst_region), df_row) in enumerate(grouped):
        # set subplots row labels on left of plot
        df_row["inter_region"] = df_row.inter_region.fillna("Direct")
        axs[plot_row_idx, 0].set_ylabel(f"{src_region} => {dst_region}")
        for plot_col_idx, (inter_region, df_plot) in enumerate(df_row.groupby("inter_region")):
            if plot_row_idx == 0:
                axs[plot_row_idx, plot_col_idx].set_title(f"Relay: {inter_region}")
            ax = axs[plot_row_idx, plot_col_idx]
            row = df_plot.iloc[0]
            runtime_df = compute_throughput_df(row["log"], row["chunk_size_mb"] * MB)
            # remove rows where runtime_upload or runtime_download is less than 0.1s
            with plt.style.context(["seaborn-white", {"font.size": 10}]):
                # plot histogram of runtime_upload, runtime_download
                # plot upload if mean upload time is greater than 0.1s
                runtime_upload_filtered = runtime_df[runtime_df.runtime_upload > 0.1]
                ax.hist(runtime_upload_filtered.runtime_upload, bins=50, label="upload", alpha=0.5)
                # plot download if mean download time is greater than 0.1s
                runtime_download_filtered = runtime_df[runtime_df.runtime_download > 0.2]
                ax.hist(runtime_df.runtime_download, bins=50, label="download", alpha=0.5)
                ax.legend()
                ax.set_xlabel("runtime (s)")
                ax.set_xlim(0, 5.0)
            pbar.update(1)

fig.patch.set_facecolor("white")
fig.tight_layout()
fig.savefig(fig_dir / "runtime_hist.png", dpi=100, bbox_inches="tight")
fig.savefig(fig_dir / "runtime_hist.pdf", bbox_inches="tight")

In [ ]:
# show upload_gbits and download_gbits distribution over all chunks over all hops
df_throughput = compute_throughput_df(df.log.iloc[0], df.chunk_size_mb.iloc[0] * MB)
with plt.style.context(["seaborn-white", {"font.size": 10}]):
    # discard rows where runtime_upload or runtime_download is less than 0.1s
    runtime_df = df_throughput.loc[(df_throughput.runtime_upload > 0.1) & (df_throughput.runtime_download > 0.1)]
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.hist(df_throughput.upload_gbits, bins=50, alpha=0.5, label="upload")
    ax.hist(df_throughput.download_gbits, bins=50, alpha=0.5, label="download")
    ax.legend()
    ax.set_xlabel("throughput (Gbps)")
    ax.set_xlim(0, 5.0)

fig.patch.set_facecolor("white")
fig.tight_layout()
fig.savefig(fig_dir / "throughput_hist.png", dpi=100, bbox_inches="tight")
fig.savefig(fig_dir / "throughput_hist.pdf", bbox_inches="tight")

In [ ]:
sub_dir = fig_dir / "bytes_complete_over_time"
sub_dir.mkdir(parents=True, exist_ok=True)

# plot grid of subplots
grouped = df.groupby(["src_region", "dst_region"])
n_rows = len(grouped.groups)
n_cols = max(len(grouped.get_group(group).inter_region.unique()) for group in grouped.groups)
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols * 4, n_rows * 4))
with tqdm(total=n_rows * n_cols) as pbar:
    for plot_row_idx, ((src_region, dst_region), df_row) in enumerate(grouped):
        # set subplots row labels on left of plot
        df_row["inter_region"] = df_row.inter_region.fillna("Direct")
        transfer_bytes = df_row.chunk_size_mb.values[0] * MB
        axs[plot_row_idx, 0].set_ylabel(f"{src_region} => {dst_region}")
        for plot_col_idx, (inter_region, df_plot) in enumerate(df_row.groupby("inter_region")):
            if plot_row_idx == 0:
                axs[plot_row_idx, plot_col_idx].set_title(f"Relay: {inter_region}")
            ax = axs[plot_row_idx, plot_col_idx]
            row = df_plot.iloc[0]
            runtime_df = compute_throughput_df(row["log"], row["chunk_size_mb"] * MB).reset_index()
            runtime_df = runtime_df[runtime_df["hop_idx"] == max(runtime_df["hop_idx"])]

            # select ['upload_complete_time'] column
            time_uploaded_df = (
                runtime_df.loc[:, ["upload_complete_time"]]
                .dropna()
                .reset_index(drop=True)
                .sort_values(by="upload_complete_time")
                .reset_index(drop=True)
            )
            # bytes_uploaded = (idx + 1) * transfer_bytes
            time_uploaded_df["bytes_uploaded"] = (time_uploaded_df.index + 1) * transfer_bytes
            if len(time_uploaded_df) > 0:
                time_uploaded_df["upload_complete_time"] = time_uploaded_df.upload_complete_time - min(
                    time_uploaded_df.upload_complete_time
                )
                if max(time_uploaded_df.bytes_uploaded) > 1 * GB:
                    with plt.style.context(["seaborn-white", {"font.size": 10}]):
                        # plot upload_complete_time vs bytes_uploaded
                        ax.plot(time_uploaded_df.upload_complete_time, time_uploaded_df.bytes_uploaded / GB, label="uploaded")
                        ax.set_xlabel("Time (s)")
                        if plot_col_idx == 0:
                            ax.set_ylabel(f"GB uploaded\n{src_region} => {dst_region}")
                        else:
                            ax.set_ylabel(f"GB uploaded")
            pbar.update(1)
# white background
fig.patch.set_facecolor("white")

In [ ]:
runtime_df